In [67]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import svm
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn import neighbors
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
import scipy
from sklearn.manifold import TSNE
from pandas.plotting import parallel_coordinates
from sklearn import decomposition
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn import svm
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn import neighbors
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
import scipy
from math import  pi, exp,log, sqrt
from scipy.stats import norm
from sklearn.manifold import TSNE
import seaborn as sn
from sklearn.datasets import make_friedman1
from sklearn.decomposition import SparsePCA
from dask import dataframe as dd

In [91]:
OP_df = pd.read_csv('dataset/Option2017_2.csv')
OP_df = OP_df.dropna()
OP_df.head(3)

,Unnamed: 0,currentDate,ExpDate,StrikePrice,Ticker,Type,Last,Implied volatility,Underlaying asset price,Expiration time
0,0,2/17/17,7/21/17,34.799999,TBNK,Call,1.06,0.13250,33.549999,0.421629
1,1,2/17/17,3/17/17,20.000000,ARR,Call,1.70,0.02022,21.709999,0.076660
2,2,2/17/17,3/17/17,22.500000,ARR,Call,0.08,0.13956,21.709999,0.076660


In [69]:
def d1(S,K,T,r,sigma):
    
    x = ( log (S/K) + ( ( r + ((sigma**2)/2) )* T  )  ) / ( sigma * ( sqrt ( T ) ) )
    return x 

def d2(S,K,T,r,sigma):
    
    x = ( log (S/K) + ( ( r - ((sigma**2)/2) )* T  )  ) / ( sigma * ( sqrt ( T ) ) )
    return x

In [70]:
df = OP_df

In [71]:
def call(S,K,T,r,sigma):

    return S  * norm.cdf( d1(S,K,T,r,sigma) ) - K * (exp(- (r*T) ) ) * norm.cdf( d2(S,K,T,r,sigma) )

def put(S,K,T,r,sigma):

    return   -norm.cdf(- d1(S,K,T,r,sigma)) * S + norm.cdf(-d2(S,K,T,r,sigma)) * K * np.exp(-r * T)

In [72]:
from scipy import optimize
import scipy.stats as si
def callvol(price,S,K,r,T):
  
    def brentt(sigma):
        d11 = d1(S,K,T,r,sigma)
        d22 = d2(S,K,T,r,sigma)
        Brent = call(S,K,T,r,sigma)
        fx = Brent - price
        return fx

    x = optimize.brentq(brentt,0.000000000001,10,maxiter=1000,full_output=True)
    return x[0] , x[1].iterations

callvol( price=0.30, S=21.709999,K=22.000000, r=0.03,T=0.172485 )

(0.10570986506314022, 7)

In [73]:
from scipy import optimize
import scipy.stats as si
def putvol(price,S,K,r,T):
  
    def brentp(sigma):
        d11 = d1(S,K,T,r,sigma)
        d22 = d2(S,K,T,r,sigma)
        Brent = put(S,K,T,r,sigma)
        fx = Brent - price
        return fx

    x = optimize.brentq(brentp,0.000000000001,10,maxiter=1000,full_output=True)
    return x[0] , x[1].iterations

#callvol( price=0.30, S=21.709999,K=22.000000, r=0.03,T=0.172485 )

In [54]:
m,iter = (callvol( 0.30 , 21.709999 , 22.000000 , 0.03, 0.172485 )) 
m

0.1057098650631396

8

In [92]:
S = np.array( OP_df["Underlaying asset price"]).astype("float")
K = np.array( OP_df["StrikePrice"]).astype("float")
r = .03
T = np.array( OP_df["Expiration time"]).astype("float")
Op = np.array( OP_df["Last"]).astype("float")
Type = np.array( OP_df["Type"]).astype("str")
OP_df.head(3)
df = OP_df
df.head(2)

,Unnamed: 0,currentDate,ExpDate,StrikePrice,Ticker,Type,Last,Implied volatility,Underlaying asset price,Expiration time
0,0,2/17/17,7/21/17,34.799999,TBNK,Call,1.06,0.13250,33.549999,0.421629
1,1,2/17/17,3/17/17,20.000000,ARR,Call,1.70,0.02022,21.709999,0.076660


In [93]:
imp = []
iter = []
for i in range( len(OP_df) ):
    if(Type[i] == "Call"):
        try:
            
            m,it = (callvol( Op[i] , S[i] , K[i] , r , T[i] )) 
            imp.append(m)
            iter.append(it)
            if(it >900):
                df = df.drop(index= (i))
        except:
            df = df.drop(index= (i))
    else:
        try:
            
            m,it = (putvol( Op[i] , S[i] , K[i] , r , T[i] )) 
            imp.append(m)
            iter.append(it)
            if(it >900):
                df = df.drop(index= (i))
        except:
            df = df.drop(index= (i))

In [94]:
df.head(5)

,Unnamed: 0,currentDate,ExpDate,StrikePrice,Ticker,Type,Last,Implied volatility,Underlaying asset price,Expiration time
0,0,2/17/17,7/21/17,34.799999,TBNK,Call,1.06,0.13250,33.549999,0.421629
2,2,2/17/17,3/17/17,22.500000,ARR,Call,0.08,0.13956,21.709999,0.076660
3,3,2/17/17,3/17/17,20.000000,ARR,Put,0.20,0.32783,21.709999,0.076660
4,4,2/17/17,3/17/17,22.500000,ARR,Put,0.95,0.27590,21.709999,0.076660
7,7,2/17/17,4/21/17,22.000000,ARR,Call,0.30,0.10719,21.709999,0.172485


In [77]:
len(OP_df)

17559

In [86]:
OP_df['imp'] = pd.Series(imp)
OP_df['iter'] = pd.Series(iter)
OP_df.head(3)

,Unnamed: 0,currentDate,ExpDate,StrikePrice,Ticker,Type,Last,Implied volatility,Underlaying asset price,Expiration time,imp,iter
0,0,2/17/17,7/21/17,34.799999,TBNK,Call,1.06,0.13250,33.549999,0.421629,0.162619,8.0
1,1,2/17/17,3/17/17,20.000000,ARR,Call,1.70,0.02022,21.709999,0.076660,0.132739,12.0
2,2,2/17/17,3/17/17,22.500000,ARR,Call,0.08,0.13956,21.709999,0.076660,0.341223,11.0


16275

In [95]:
len(df)

16275

In [80]:
df.head(2)

,Unnamed: 0,currentDate,ExpDate,StrikePrice,Ticker,Type,Last,Implied volatility,Underlaying asset price,Expiration time


,bisection
0,0.032722
1,81.930349
2,0.219175


,bisection,Brent
0,0.031976,0.032722
1,47.563301,81.930349
2,0.249562,0.219175


In [96]:
df.to_csv('EuropeanOption2017.csv', index=False)

In [10]:
df.head(3)

,Unnamed: 0,currentDate,ExpDate,StrikePrice,Ticker,Type,Last,Implied volatility,Underlaying asset price,Expiration time
0,0,2/17/17,7/21/17,34.799999,TBNK,Call,1.06,0.13250,33.549999,0.421629
1,1,2/17/17,3/17/17,20.000000,ARR,Call,1.70,0.02022,21.709999,0.076660
2,2,2/17/17,3/17/17,22.500000,ARR,Call,0.08,0.13956,21.709999,0.076660


In [13]:
df = df.drop(index=1)
df.head(3)

,Unnamed: 0,currentDate,ExpDate,StrikePrice,Ticker,Type,Last,Implied volatility,Underlaying asset price,Expiration time
0,0,2/17/17,7/21/17,34.799999,TBNK,Call,1.06,0.13250,33.549999,0.421629
2,2,2/17/17,3/17/17,22.500000,ARR,Call,0.08,0.13956,21.709999,0.076660
3,3,2/17/17,3/17/17,20.000000,ARR,Put,0.20,0.32783,21.709999,0.076660
